In [ ]:
'''Python Introductions provided by FireWorks'''

In [ ]:
'''How FireWorks performs calculations'''
from fireworks import Firework, FWorker, LaunchPad
from fireworks.core.rocket_launcher import launch_rocket
from fw_tutorials.firetask.addition_task import AdditionTask

loglvl = "CRITICAL"
# set up the LaunchPad and reset it
launchpad = LaunchPad(strm_lvl=loglvl)
# launchpad.reset('', require_password=False)

# create the Firework consisting of a custom "Addition" task
firework = Firework(AdditionTask(), 
                    spec={"input_array": [1, 2]})

# store workflow and launch it locally
launchpad.add_wf(firework)
launch_rocket(launchpad, FWorker(), strm_lvl=loglvl)

In [ ]:
'''How to define a FireWork's Workflow with PyTask'''
from fireworks import Firework, Workflow, FWorker, LaunchPad, PyTask
from fireworks.core.rocket_launcher import rapidfire
launchpad = LaunchPad(strm_lvl=loglvl)

# FireWorks needs to be able to find functions inside your python path, find the module name for this file
mod_name = __name__

def print_message(to_print):
    print(to_print)

# define four individual FireWorks used in the Workflow
task1 = PyTask({"func": mod_name+".print_message", "args": ["Ingrid is the CEO."]})
task2 = PyTask({"func": mod_name+".print_message", "args": ["Jill is a manager."]})
task3 = PyTask({"func": mod_name+".print_message", "args": ["Jack is a manager."]})
task4 = PyTask({"func": mod_name+".print_message", "args": ["Kip is an intern."]})

fw1 = Firework(task1, name="PyTask1")
fw2 = Firework(task2, name="PyTask2" )
fw3 = Firework(task3, name="PyTask3" )
fw4 = Firework(task4, name="PyTask4" )

# assemble Workflow from FireWorks and their connections by id
workflow = Workflow([fw1, fw2, fw3, fw4], 
                    {fw1: [fw2, fw3], fw2: [fw4], fw3: [fw4]},
                    name="PyTaskExample"
                   )

# store workflow and launch it locally
launchpad.add_wf(workflow)
rapidfire(launchpad, FWorker(), strm_lvl=loglvl)

# store workflow and launch it locally
launchpad.add_wf(workflow)
launch_rocket(launchpad, FWorker(), strm_lvl=loglvl)
launch_rocket(launchpad, FWorker(), strm_lvl=loglvl)
launch_rocket(launchpad, FWorker(), strm_lvl=loglvl)
launch_rocket(launchpad, FWorker(), strm_lvl=loglvl)

In [ ]:
'''Defining a dynamic Workflow with PyTasks'''
from fireworks import Firework, FWorker, LaunchPad, PyTask, FWAction
from fireworks.core.rocket_launcher import rapidfire
from fw_tutorials.dynamic_wf.fibadd_task import FibonacciAdderTask

# FireWorks needs to be able to find functions inside your python path, find the module name for this file
mod_name = __name__

# Define a new python function that will be used in a PyTask
def fibonacci_adder(num1, num2, stop_point=0):
    if num1 + num2 < stop_point:
        new_fw = Firework(PyTask({'func': mod_name+'.fibonacci_adder', 'args': [num2, num1 + num2, stop_point]}))
        print(f"The current number is {num1+num2}")
        # Return a FWAction that will append a new Firework at the end of the Workflow
        return FWAction(additions=new_fw)
    else:
        print(f"The limit was reached, the next number is {num1+num2}")

launchpad = LaunchPad(strm_lvl=loglvl, port=27018)
firework = Firework(PyTask({"func":mod_name+'.fibonacci_adder', "args":[0, 1, 1000]}))
launchpad.add_wf(firework)
# rapidfire will keep running Fireworks until there are no ready jobs in the LaunchPad
rapidfire(launchpad, FWorker(), strm_lvl=loglvl)

In [ ]:
'''How can we use Fireworks for electronic structure calculaions?'''

In [ ]:
# Import modules from ase and fireworks
import os
import numpy as np

from ase.build import bulk
from ase.calculators.emt import EMT
from ase.db.core import connect

from fireworks import Firework, FWorker, LaunchPad, PyTask, FWAction, Workflow
from fireworks.core.rocket_launcher import launch_rocket, rapidfire

# Minimal hilde inputs to make dictionary conversion easier
from hilde.structure.structure import pAtoms, dict2patoms, patoms2dict
from hilde.helpers.hash import hash_atoms

mod_name = __name__

In [ ]:
# Define a single point calculation calculate wrapper
def calculate(atoms_dict):
    at = dict2patoms(atoms_dict)
    at.calc.calculate(at, properties=['forces'])
    # update_spec modifies the spec of the current Firework's children to include new information with
    # the new/updated spec keys set to the keys in the dict
    return FWAction(update_spec={"calc_atoms": patoms2dict(at)})

In [ ]:
# Intialize Structures
ni = pAtoms(bulk('Ni', cubic=True))
ni.set_calculator(EMT())
ni_dict = patoms2dict(ni)

ni_hash, calc_hash = hash_atoms(ni)

In [ ]:
# To pass things to Fireworks they need to be in json string form or an object that can be easily
# converted to a json string (like a dict)
# Function are passed by their module_name.file_name
ft_calc = PyTask({"func": mod_name+".calculate", "args": [ni_dict]})

In [ ]:
# Add a firework with the task to define a job
wf = Workflow([Firework([ft_calc])])
launchpad = LaunchPad()
launchpad.add_wf(wf)
# launch_rocket computes a single Firework
launch_rocket(launchpad, strm_lvl=loglvl)

In [ ]:
# Once something is inside Fireworks it can only be accessed through it's database or a database/file
# the user specified inside the tasks
# Getting the results from FireWorks
fw_spec = launchpad.get_fw_dict_by_id(wf.root_fw_ids[0])
forces = fw_spec['launches'][0]['action']['update_spec']['calc_atoms']['forces']
print(f"Atomic forces are:\n{np.array(forces)}")

In [ ]:
# Define a function that will add a new structure calculated within Fireworks to an ase databse
def calc_to_db(db_path, atoms_dict):
    db = connect(db_path)
    at = dict2patoms(atoms_dict)
    at.calc.atoms = at
    atoms_hash, calc_hash = hash_atoms(at)
    selection = [("atoms_hash", "=", atoms_hash), ("calc_hash", "=", calc_hash)]
    # Try to update the database if the material is already present, if not add it to the database
    try:
        rows = list(db.select(selection=selection))
        if not rows:
            raise KeyError()
        for row in rows:
            db.update(row.id, at, atoms_hash=atoms_hash, calc_hash=calc_hash)
    except KeyError:
        db.write(at, atoms_hash=atoms_hash, calc_hash=calc_hash)
    return FWAction()

In [ ]:
# Set up database
db_path = (os.getcwd() + '/test.db')
print(f'database: {db_path}')
# Set up a Workflow where each FireTask has its own Firework
fw_calc = Firework(PyTask({"func": mod_name+".calculate", "args": [ni_dict]}))
# Args and Inputs are broken up into two objects (Can be up to three) 
#     args: Positional arguments that are given to Fireworks from outside its current spec
#     inputs: Postiional arguments that are given to Fireworks from inside its current spec
#             pased as a string to the spec key
#     kwargs: dict describing keyword arguments taken from outside current spec (none in this example)
#     Function call fxn_name(args, inputs, kwargs)
fw_to_db = Firework(PyTask({"func":mod_name+".calc_to_db", "args":[db_path], "inputs":["calc_atoms"]}))
# Workflows defined by list of Fireworks, and a dict describing the links between each Firework
wf = Workflow([fw_calc, fw_to_db], {fw_calc:[fw_to_db]})
launchpad.add_wf(wf)
# Launch 2 rockets to complete the workflow
rapidfire(launchpad, nlaunches=2, strm_lvl=loglvl)

2018-11-13 10:35:24,752 INFO Created new dir /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-24-752037
2018-11-13 10:35:24,753 INFO Launching Rocket
2018-11-13 10:35:24,773 INFO RUNNING fw_id: 3 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-24-752037
2018-11-13 10:35:24,778 INFO Task started: PyTask.
2018-11-13 10:35:24,805 INFO Task completed: PyTask 
2018-11-13 10:35:24,816 INFO Rocket finished
2018-11-13 10:35:24,819 INFO Created new dir /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-24-819367
2018-11-13 10:35:24,820 INFO Launching Rocket
2018-11-13 10:35:24,828 INFO RUNNING fw_id: 2 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-24-819367
2018-11-13 10:35:24,832 INFO Task started: PyTask.
2018-11-13 10:35:24,862 INFO Task completed: PyTask 
2018-11-13 10:35:24,869 INFO Rocket finished


In [ ]:
# See the results with the database access
db = connect(db_path)
at = db.get_atoms(calc_hash=calc_hash, atoms_hash=ni_hash, attach_calculator=True)
row = list(db.select(selection=[("atoms_hash", "=", ni_hash)], columns=["forces"]))[0]
print(f"Atomic forces from the new atoms are: \n{at.get_forces()}")
print(f"Atomic forces from the row is: \n{row.forces}")

In [ ]:
'''Building up a Workflow for phonon calculations'''

In [ ]:
# Import modules from ase and fireworks
import os
import numpy as np

from ase.build import bulk
from ase.calculators.emt import EMT

from fireworks import Firework, FWorker, LaunchPad, PyTask, FWAction, Workflow
from fireworks.core.rocket_launcher import launch_rocket, rapidfire

# Minimal hilde inputs to make dictionary conversion easier
from hilde.structure.structure import pAtoms, dict2patoms, patoms2dict
from hilde.helpers.hash import hash_atoms

# Modified ASE database for phonon calculations
from hilde.phonon_db.row import phonon_to_dict, PhononRow
from hilde.phonon_db.phonon_db import connect as connect_ph

from phonopy import Phonopy
from phonopy.structure.atoms import PhonopyAtoms

mod_name = __name__

In [ ]:
# Utility function to convert from ASE atoms to Phonopy Atoms
def to_phonopy_atoms(atoms):
    phonopy_atoms = PhonopyAtoms(
        symbols=atoms.get_chemical_symbols(),
        cell=atoms.get_cell(),
        masses=atoms.get_masses(),
        positions=atoms.get_positions(wrap=True),
    )
    return phonopy_atoms

In [ ]:
# Define calculate with a mod_spec instead of update_spec
# mod_spec allows for appending items to the end of lists
def calculate(atoms_dict):
    print("Single point calculation")
    at = dict2patoms(atoms_dict)
    at.calc.calculate(at, properties=['forces'])
    # mod_spec allows the user to add elements at the end of a list stored inside the spec
    # with key defined by the input dict ("calc_forces" in this key)
    return FWAction(mod_spec=[{'_push': {"calc_forces": patoms2dict(at)}}])

In [ ]:
# Function to setup multiple calculate Fireworks
def calculate_multiple(atom_dicts):
    print("Appending calculations to the workflow")
    firework_detours = []
    for i, cell in enumerate(atom_dicts):
        task = PyTask({"func": mod_name+".calculate",
                       "args": [cell]})
        firework_detours.append(Firework(task, name=f"calc_{i}"))
    # detours appends the new Fireworks as the children of the current one and moves the
    # previous children of the Firework to be the children of the new Fireworks
    return FWAction(detours=firework_detours)

In [ ]:
# Function to calculate the force constants from the Fireworks spec
def get_fcs(phonon_dict, atoms_ideal, calc_atoms):
    print("Getting the force constants")
    atoms = dict2patoms(atoms_ideal)
    disp_cells = [dict2patoms(ca) for ca in calc_atoms]
    # Sort the database list as there is no gaurentee that the calculations will finish in order
    disp_cells = sorted(disp_cells, key=lambda x: x.info['disp_num'])
    phonon = PhononRow(phonon_dict).to_phonon()
    phonon.generate_displacements(distance=0.01)
    phonon.set_forces([cell.get_forces() for cell in disp_cells])
    phonon.produce_force_constants()
    return FWAction(update_spec={"phonon_dict": phonon_to_dict(phonon)})


In [ ]:
# Redefine the databse addition function to work with phonopy objects
def calc_to_db(db_path, sc_dict, phonon_dict):
    print("Adding results to the database")
    db = connect_ph(db_path)
    at = dict2patoms(sc_dict)
    at.calc.atoms = at
    atoms_hash, calc_hash = hash_atoms(at)
    selection = [("atoms_hash", "=", atoms_hash), 
                 ("calc_hash", "=", calc_hash), 
                 ("sc_matrix_2", "=", phonon_dict["sc_matrix_2"])]
    try:
        rows = list(db.select(selection=selection))
        if not rows:
            raise KeyError()
        for row in rows:
            db.update(row.id, phonon_dict, atoms_hash=atoms_hash, calc_hash=calc_hash)
    except KeyError:
        db.write(phonon_dict, atoms_hash=atoms_hash, calc_hash=calc_hash)
    return FWAction()

In [ ]:
#Initialize structures
smatrix = np.array([3, 0, 0, 0, 3, 0, 0, 0, 3]).reshape(3,3)
db_path = os.getcwd() + "test_ph.db"

# Intialize Structures
al = pAtoms(bulk('Al', 'fcc'))
al.set_calculator(EMT())
al_dict = patoms2dict(al)
al_hash, calc_hash = hash_atoms(al)

ph_atoms = to_phonopy_atoms(al)

phonon = Phonopy(ph_atoms, supercell_matrix=smatrix, symprec=1e-5, is_symmetry=True, factor=15.633302)
phonon_dict = phonon_to_dict(phonon)
pc_dict = patoms2dict(al)
phonon.generate_displacements(distance=0.01)
scs = phonon.get_supercells_with_displacements()
sc_dicts = []
for i, sc in enumerate(scs):
    scs[i] = pAtoms(phonopy_atoms=sc)
    scs[i].info['disp_num'] = i
    scs[i].calc = al.calc
    sc_dicts.append(patoms2dict(scs[i]))

In [ ]:
# Create Firetasks for each part of the calculation
ft_initialize = PyTask({"func": mod_name + ".calculate_multiple", "args":[sc_dicts]})
ft_get_fc = PyTask({"func": mod_name + ".get_fcs", "args":[phonon_dict, pc_dict], "inputs":["calc_forces"]})
ft_calc_to_db = PyTask({"func": mod_name + ".calc_to_db", "args":[db_path, pc_dict], "inputs":["phonon_dict"]})

In [ ]:
# Initialize the Fireworks
fw_initialize = Firework(ft_initialize)
fw_get_fc = Firework(ft_get_fc)
fw_calc_to_db = Firework(ft_calc_to_db)
# Note there is no need to define links for the Force evaluation calculations
wf = Workflow([fw_initialize, fw_get_fc, fw_calc_to_db], {fw_initialize:[fw_get_fc], fw_get_fc: [fw_calc_to_db]})

2018-11-13 10:35:25,324 INFO Created new dir /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-25-324373
2018-11-13 10:35:25,325 INFO Launching Rocket
2018-11-13 10:35:25,335 INFO RUNNING fw_id: 6 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-25-324373
2018-11-13 10:35:25,340 INFO Task started: PyTask.
2018-11-13 10:35:25,341 INFO Task completed: PyTask 
2018-11-13 10:35:25,360 INFO Rocket finished
2018-11-13 10:35:25,364 INFO Created new dir /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-25-364235
2018-11-13 10:35:25,364 INFO Launching Rocket
2018-11-13 10:35:25,374 INFO RUNNING fw_id: 7 in directory: /home/knoop/local/hilde/examples/fireworks/fireworks_tutorial/launcher_2018-11-13-09-35-25-364235
2018-11-13 10:35:25,380 INFO Task started: PyTask.
2018-11-13 10:35:25,544 INFO Task completed: PyTask 
2018-11-13 10:35:25,569 INFO Rocket finished
2018-11-

In [ ]:
launchpad = LaunchPad()
launchpad.reset('', require_password=False)
launchpad.add_wf(wf)
# nlaunches=0 means keep running Fireworks until none are needed
rapidfire(launchpad, nlaunches=0, strm_lvl="CRITICAL")

In [ ]:
#Access the database to check the results
db = connect_ph(db_path)
row = list(db.select(selection=[("atoms_hash", "=", al_hash), 
                                ("sc_matrix_2", "=", list(smatrix.flatten()))], 
                     columns=["fc_2"]))[0]
print(f"The force constants are:\n{row.get('fc_2')}")